In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [ ]:
tag_users = pd.read_csv("data/tag_users.csv")
tag_questions = pd.read_csv("data/tag_questions.csv")
tags = pd.read_csv("data/tags.csv")
professionals = pd.read_csv("data/professionals.csv")
answers = pd.read_csv("data/answers.csv")
emails1 = pd.read_csv("data/emails1.csv")
emails2 = pd.read_csv("data/emails2.csv")
emails = pd.concat([emails1,emails2])
questions = pd.read_csv("data/questions.csv")
matches1 = pd.read_csv("data/matches1.csv")
matches2 = pd.read_csv("data/matches2.csv")
matches3 = pd.read_csv("data/matches3.csv")
matches = pd.concat([matches1,matches2,matches3])
answers_scores = pd.read_csv("data/answer_scores.csv")
group_members = pd.read_csv("data/group_memberships.csv")
schools_members = pd.read_csv("data/school_memberships.csv")

In [ ]:
def get_user_tags(user_id):
    user_tags = tag_users[tag_users["tag_users_user_id"] == user_id]["tag_users_tag_id"]
    tags_names =tags.loc[tags["tags_tag_id"].isin(user_tags)]
    return(tags)

In [ ]:
def get_user_answers(user_id):
    all_answers = answers.loc[answers["answers_author_id"]==user_id]
    return all_answers

In [ ]:
def match_email_question(email_date):
    f = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S UTC+0000")
    b  =f(email_date)
    t = questions["questions_date_added"].apply(f)
    return questions.iloc[(b-t).abs().argsort()[:1]]

In [ ]:
immediate_emails = emails[emails["emails_frequency_level"]=="email_notification_immediate"]
immediate_matches = matches[matches["matches_email_id"].isin(immediate_emails["emails_id"])]
immediate_emails = immediate_emails[immediate_emails["emails_id"].isin(matches["matches_email_id"])]

immediate_emails = immediate_emails.rename(columns={'emails_id':"id"})
immediate_matches = immediate_matches.rename(columns={'matches_email_id':"id"})

In [ ]:
emails_questions = pd.merge(immediate_emails, immediate_matches, how="left", on="id")
emails_questions = emails_questions.rename(columns = {"matches_question_id": "question_id", "emails_recipient_id": "professional_id"})
answers = answers.rename(columns = {"answers_question_id": "question_id", "answers_author_id": "professional_id"})

In [ ]:
full_data = pd.merge(emails_questions, answers,how='left',  on=["professional_id","question_id" ])
target = full_data['answers_id'].fillna(0)
target = pd.to_numeric(target, errors='coerce').fillna(1).astype(int)
full_data["q_answered"] = target
full_data = full_data.rename(columns = {"id": "email_id"})
date_vectorizer = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S UTC+0000")
full_data["emails_date_sent"] = full_data["emails_date_sent"].apply(date_vectorizer)
indices = full_data["answers_date_added"][full_data["answers_date_added"].notnull()].index.values
full_data.loc[indices, "answers_date_added"] = full_data["answers_date_added"][full_data["answers_date_added"].notnull()].apply(date_vectorizer)
full_data["time_taken"] =  pd.to_datetime(full_data["answers_date_added"]) - pd.to_datetime(full_data["emails_date_sent"])
indices2 = full_data["time_taken"][full_data["time_taken"] < datetime.timedelta( days=0)].index.values
full_data.loc[indices2, "time_taken"]=datetime.timedelta(days=0, seconds=0,minutes=0, hours=0 )

In [ ]:
response = full_data.groupby("professional_id").mean().drop(columns = ["email_id"]).reset_index()
professionals_dataset = pd.DataFrame(columns = ["professional_id"])
professionals_dataset["professional_id"] = professionals["professionals_id"]
professionals_dataset = pd.merge(professionals_dataset, response,how='left',  on=["professional_id" ])
professionals_dataset = professionals_dataset.rename(columns = {"q_answered": "response_rate"})
grouped = full_data.groupby('professional_id')["time_taken"]
time_mean = grouped.apply(lambda x: np.mean(x))
professionals_dataset = pd.merge(professionals_dataset, time_mean,how='left',  on=["professional_id" ])
professionals_dataset=  professionals_dataset.rename(columns ={"time_taken": "avg_time_taken"})
answers_count = answers["professional_id"].value_counts().reset_index().rename(columns = {"professional_id":"number_q_answered","index":"professional_id",  })
professionals_dataset = pd.merge(professionals_dataset, answers_count,how='left',  on=["professional_id" ])
all_tags = pd.merge(tag_users.rename(columns = {"tag_users_tag_id": "tag_id"}),tags.rename(columns = {"tags_tag_id": "tag_id"}),how='left',  on=["tag_id" ])
foll_tags = all_tags.groupby('tag_users_user_id')['tags_tag_name'].apply(list).reset_index(name='following_tags').rename(columns = {"tag_users_user_id": "professional_id"})
professionals_dataset = pd.merge(professionals_dataset, foll_tags,how='left',  on=["professional_id" ])
a = pd.merge(answers["question_id"].reset_index(), tag_questions.rename(columns = {"tag_questions_question_id": "question_id"}),how='left',  on=["question_id" ])
b = pd.merge(a.rename(columns = {"tag_questions_tag_id":"tags_tag_id"}), tags,how='left',  on=["tags_tag_id" ]).drop(columns = ["index", "tags_tag_id"])
c = pd.merge(b, answers,how='left',  on=["question_id" ]).drop(columns = ["answers_id", "answers_date_added", "answers_body", "question_id"])
d = c.groupby("professional_id")["tags_tag_name"].apply(list).reset_index()
professionals_dataset = pd.merge(professionals_dataset, d.rename(columns = {"tags_tag_name": "prev_q_tags"}),how='left',  on=["professional_id" ])
prof_score = pd.merge(answers[["professional_id","answers_id" ]], answers_scores.rename(columns = {"id":"answers_id"}),how='left',  on=["answers_id" ]).drop(columns = ["answers_id"])
prof_score.groupby("professional_id").mean().reset_index()
professionals_dataset = pd.merge(professionals_dataset,prof_score,how='left',  on=["professional_id" ]).rename(columns = {"score":"avg_ansrs_score"})
prof_grp = group_members["group_memberships_user_id"].value_counts().reset_index().rename(columns = {"index":"professional_id", "group_memberships_user_id":"num_groups"})
professionals_dataset = pd.merge(professionals_dataset,prof_grp,how='left',  on=["professional_id" ])
prof_schl = schools_members["school_memberships_user_id"].value_counts().reset_index().rename(columns = {"index":"professional_id", "school_memberships_user_id":"num_schools"})
professionals_dataset = pd.merge(professionals_dataset,prof_schl,how='left',  on=["professional_id" ])
prof_emls = immediate_emails["emails_recipient_id"].value_counts().reset_index().rename(columns = {"index":"professional_id", "emails_recipient_id":"num_emails"})
professionals_dataset = pd.merge(professionals_dataset,prof_emls,how='left',  on=["professional_id" ])
professionals_dataset["answrs_emails_ratio"] = professionals_dataset["number_q_answered"] / professionals_dataset["num_emails"]


In [ ]:
professionals_dataset

,professional_id,Unnamed: 0_x,Unnamed: 0_y,response_rate,avg_time_taken,number_q_answered,following_tags,prev_q_tags,avg_ansrs_score,num_groups,num_schools,num_emails,answrs_emails_ratio
0,9ced4ce7519049c0944147afb75a8ce3,NaN,NaN,NaN,NaT,1.0,NaN,"[consulting, resume, consulting, resume, consu...",5.0,NaN,NaN,NaN,NaN
1,f718dcf6d2ec4cb0a52a9db59d7f9e67,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0c673e046d824ec0ad0ebe012a0673e4,329.342857,2.402384e+06,0.0,NaT,39.0,"[consulting, education, consulting, education,...","[experience, healthcare, doctor, anesthesiolog...",1.0,NaN,NaN,70.0,0.557143
3,0c673e046d824ec0ad0ebe012a0673e4,329.342857,2.402384e+06,0.0,NaT,39.0,"[consulting, education, consulting, education,...","[experience, healthcare, doctor, anesthesiolog...",0.0,NaN,NaN,70.0,0.557143
4,0c673e046d824ec0ad0ebe012a0673e4,329.342857,2.402384e+06,0.0,NaT,39.0,"[consulting, education, consulting, education,...","[experience, healthcare, doctor, anesthesiolog...",3.0,NaN,NaN,70.0,0.557143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68186,9f267950ab8e43e2afcf564327049297,NaN,NaN,NaN,NaT,NaN,[broadcast-media],NaN,NaN,NaN,NaN,NaN,NaN
68187,4a7e1e5dd884488da283e1cab6ad11e0,NaN,NaN,NaN,NaT,NaN,"[telecommunications, firstnet, #inbuildingsolu...",NaN,NaN,NaN,NaN,NaN,NaN
68188,ea75c5fce38348e0a151c3c346929e6a,NaN,NaN,NaN,NaT,NaN,"[computer-software, electrical-engineering, ha...",NaN,NaN,NaN,NaN,NaN,NaN
68189,34f6222c3c02480ca2df8a3e4ba878d0,NaN,NaN,NaN,NaT,NaN,[telecommunications],NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
professionals_dataset["answrs_emails_ratio"][professionals_dataset["answrs_emails_ratio"]> 1]

309       2.333333
310       2.333333
311       2.333333
312       2.333333
313       2.333333
           ...    
63384    13.000000
63385    13.000000
63386    13.000000
63387    13.000000
63388    13.000000
Name: answrs_emails_ratio, Length: 7622, dtype: float64

In [ ]:
answers.head(1)

,answers_id,professional_id,question_id,answers_date_added,answers_body
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...


In [ ]:
emails.head(1)

,Unnamed: 0,emails_id,emails_recipient_id,emails_date_sent,emails_frequency_level
0,0,2337714,0c673e046d824ec0ad0ebe012a0673e4,2018-12-07 01:05:40 UTC+0000,email_notification_daily


In [ ]:
matches.head(1)

,Unnamed: 0,matches_email_id,matches_question_id
0,0,1721939,332a511f1569444485cf7a7a556a5e54


In [ ]:
match_proff = pd.merge(matches.rename(columns = {"matches_email_id":"emails_id"}), emails,how='left',  on=["emails_id" ])
match_proff.head(1)

,Unnamed: 0_x,emails_id,matches_question_id,Unnamed: 0_y,emails_recipient_id,emails_date_sent,emails_frequency_level
0,0,1721939,332a511f1569444485cf7a7a556a5e54,1303869,172d6cac9efd4f41929430b163854b6b,2018-03-25 01:03:06 UTC+0000,email_notification_daily


In [ ]:
answers.head(1)

,answers_id,professional_id,question_id,answers_date_added,answers_body
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesti...


In [ ]:
matches_questions = pd.merge(answers, match_proff.rename(columns = {"emails_recipient_id":"professional_id" }),how='left',  on=["professional_id" ])[["answers_id", "professional_id","question_id","emails_id" ]]


In [ ]:
matches_questions["emails_id"] = matches_questions["emails_id"].fillna(0)
indices = matches_questions["emails_id"][matches_questions["emails_id"]>0].index.values
matches_questions.loc[indices,"emails_id" ]=1

In [ ]:
matches_questions.rename(columns = {"emails_id": "is_matched"})

,answers_id,professional_id,question_id,is_matched
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1.0
1,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1.0
2,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1.0
3,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1.0
4,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,1.0
...,...,...,...,...
26521409,563bc22e715e42bfb4b6ef9c3a95fb23,5750e3d592cd4b0486ec4e7eb01c29ad,1b1ebe9afc5046ccbb328704e88e5cc0,1.0
26521410,563bc22e715e42bfb4b6ef9c3a95fb23,5750e3d592cd4b0486ec4e7eb01c29ad,1b1ebe9afc5046ccbb328704e88e5cc0,1.0
26521411,563bc22e715e42bfb4b6ef9c3a95fb23,5750e3d592cd4b0486ec4e7eb01c29ad,1b1ebe9afc5046ccbb328704e88e5cc0,1.0
26521412,563bc22e715e42bfb4b6ef9c3a95fb23,5750e3d592cd4b0486ec4e7eb01c29ad,1b1ebe9afc5046ccbb328704e88e5cc0,1.0


In [ ]:
matrh

#NLP Functions

In [ ]:
#Functions for NLP

def lowercase(input):
  return input.lower()

def remove_punctuation(input):
  return input.translate(str.maketrans('','', string.punctuation))

def remove_whitespaces(input):
  return " ".join(input.split())
  
def remove_html_tags(input):
    soup = BeautifulSoup(input, "html.parser")
    stripped_input = soup.get_text(separator=" ")
    return stripped_input

def tokenize(input):
  return word_tokenize(input)

def remove_stop_words(input):
  input = word_tokenize(input)
  return [word for word in input if word not in stopwords.words('english')]

def lemmatize(input):
  lemmatizer=WordNetLemmatizer()
  input_str=word_tokenize(input)
  new_words = []
  for word in input_str:
    new_words.append(lemmatizer.lemmatize(word))
  return ' '.join(new_words)

#Function that calls all other functions together to perform NLP on a given text
def nlp_pipeline(input):
  return lemmatize(' '.join(remove_stop_words(remove_whitespaces(remove_punctuation(remove_html_tags(lowercase(input)))))))

#Topic Modeling for individual questions

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

#Turn tags into a set for faster checking of whether a tag exists or not
unique_tags = set(tags['tags_tag_name'])

def find_topics(question_body):
  """
  Function that takes a question as an input, and finds the two most important topics/tags
  If the found topics exist in the already existing database of tags, we add these tags
  to the professional who answered the question
  """

  text = nlp_pipeline(question_body)
  count_vectorizer = CountVectorizer(stop_words='english')
  count_data = count_vectorizer.fit_transform([text])
  # One topic that has an avg of two words because most questions had 1/2 tags
  number_topics = 1
  number_words = 2
  # Create and fit the LDA model
  lda = LDA(n_components=number_topics, n_jobs=-1)
  lda.fit(count_data)

  words = count_vectorizer.get_feature_names()

  #Get topics from model. They are represented as a list e.g. ['military','army']
  topics = [words[i] for i in topic.argsort()[:-number_words - 1:-1] for topic_idx, topic in enumerate(lda.components_)]

  #Only use topics for which a tag already exists
  existing_topics = []
  for tag in topics:
    if tag in unique_tags:
      existing_topics.append(tag)

  return existing_topics